## Ejercicio 3: Simulación de tareas de larga duración
Utilizando el ejemplo de lazy_printer, crea un simulador para tareas de larga duración que afecten la respuesta del servidor basado en la complejidad de la tarea.

Pasos:

- Define varias corutinas que simulen diferentes tiempos y complejidades de tareas (por ejemplo, 
- cálculos matemáticos complejos o procesamiento de texto).
- Utiliza asyncio.wait para manejar estas tareas de forma concurrente en el cliente y envía estas tareas al servidor.
- Añade lógica en el servidor para responder a estas tareas con diferentes tiempos de espera basados en su complejidad.

## servidor

In [1]:
import asyncio
import pickle
import nest_asyncio
import random
import time

nest_asyncio.apply()

# Diccionario para almacenar el estado de cada cliente
client_states = {}

async def handle_submit_job(reader, writer):
    client_id = id(writer)
    job_id = client_states.get(client_id, [0])[-1] + 1
    client_states[client_id] = client_states.get(client_id, []) + [job_id]
    writer.write(job_id.to_bytes(4, 'little'))
    await writer.drain()
    data_length = int.from_bytes(await reader.read(4), 'little')
    data = pickle.loads(await reader.read(data_length))
    
    # Simular tarea de larga duración
    complexity = random.randint(1, 5)
    sleep_time = complexity * 0.5
    print(f"Procesando tarea de complejidad {complexity} (duración: {sleep_time:.2f} segundos)")
    await asyncio.sleep(sleep_time)
    result = sum(data)
    client_states[client_id][-1] = result

async def handle_get_results(reader, writer):
    client_id = id(writer)
    job_id = int.from_bytes(await reader.read(4), 'little')
    result = client_states[client_id][job_id - 1]
    result_data = pickle.dumps(result)
    writer.write(len(result_data).to_bytes(4, 'little'))
    writer.write(result_data)
    await writer.drain()

async def accept_requests(reader, writer):
    try:
        op = await reader.read(1)
        if op:
            if op[0] == 0:
                await handle_submit_job(reader, writer)
            elif op[0] == 1:
                await handle_get_results(reader, writer)
    except (IndexError, asyncio.IncompleteReadError, pickle.UnpicklingError) as e:
        print(f"Error en accept_requests: {e}")
        writer.close()
        await writer.wait_closed()

async def main():
    server = await asyncio.start_server(accept_requests, '127.0.0.1', 1936)
    async with server:
        await server.serve_forever()

asyncio.run(main())


Procesando tarea de complejidad 1 (duración: 0.50 segundos)
